To get started, answer the following questions using just the fullsample.csv jobs dataset:

Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

In [5]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import time
import re
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
pd.options.display.max_rows=1000


In [6]:
#Job list with only completed jobs
fullsample_completed = pd.read_csv('fullsample_completed')

In [7]:
#cleaning table up

#changing to datetime
fullsample_completed['BEGIN'] = pd.to_datetime(fullsample_completed['BEGIN'])
fullsample_completed['END'] = pd.to_datetime(fullsample_completed['END'])

# changing REQTIME and USEDTIME from string to time type
fullsample_completed['REQTIME'] = fullsample_completed['REQTIME'].str.replace("-", " days ")
fullsample_completed['USEDTIME'] = fullsample_completed['USEDTIME'].str.replace("-", " days ")
fullsample_completed['REQTIME'] = pd.to_timedelta(fullsample_completed['REQTIME'])
fullsample_completed['USEDTIME'] = pd.to_timedelta(fullsample_completed['USEDTIME'])

#extract unit to new column and turn REQMEM into int type for later calculations 
fullsample_completed['Unit'] = fullsample_completed['REQMEM'].str.extract(r'([a-zA-Z]+)')
fullsample_completed['REQMEM'] =fullsample_completed['REQMEM'].str.split(r'([a-zA-Z]+)').str[0]
fullsample_completed['REQMEM'] = fullsample_completed['REQMEM'].astype('int')

#extract unit, renme column, turn into float type
fullsample_completed['USEDMEM'] = fullsample_completed['USEDMEM'].str.split(r'([a-zA-Z]+)').str[0]
fullsample_completed['USEDMEM'] = fullsample_completed['USEDMEM'].astype('float')
fullsample_completed.rename(columns={'USEDMEM': 'USEDMEM (Mn)'},inplace=True)

#move Unit column
unit = fullsample_completed['Unit']
fullsample_completed = fullsample_completed.drop(columns=['Unit'])
fullsample_completed.insert(loc=5, column='Unit', value=unit)

In [8]:
fullsample_completed.to_pickle('PICKLE')

In [9]:
PICKLE = pd.read_pickle("PICKLE")

In [11]:
PICKLE

JOBID                    object
STATE                    object
BEGIN            datetime64[ns]
END              datetime64[ns]
REQMEM                    int32
Unit                     object
USEDMEM (Mn)            float64
REQTIME         timedelta64[ns]
USEDTIME        timedelta64[ns]
NODES                     int64
CPUS                      int64
PARTITION                object
EXITCODE                 object
dtype: object